In [1]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from qiskit import Aer, transpile, assemble, execute
from qiskit.providers.aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector, Operator

# Define the parameters
mu = 1.0
N_sites = 4
interaction_strength = mu / (2 * N_sites)
tolerance = 5e-1
# Define time steps
times = np.linspace(0, 10, 1000)  # Extending the time range to 10
τ = times[1] - times[0]  # Time step
shots = 5024667
backend = Aer.get_backend('qasm_simulator')

# Define the Hamiltonian H_SI for 4 qubits with all-to-all interactions
# H_SI = 0
H_SI = np.zeros((2**N_sites, 2**N_sites), dtype=complex)
for i in range(N_sites-1):
    for j in range(i + 1, N_sites):
        XX = Pauli(f'{"I"*i}X{"I"*(j-i-1)}X{"I"*(N_sites-j-1)}').to_matrix()
        YY = Pauli(f'{"I"*i}Y{"I"*(j-i-1)}Y{"I"*(N_sites-j-1)}').to_matrix()
        ZZ = Pauli(f'{"I"*i}Z{"I"*(j-i-1)}Z{"I"*(N_sites-j-1)}').to_matrix()
        H_SI += interaction_strength * (XX + YY + ZZ)
        
def evolve_and_measure_circuit(time, H_SI):
    U = Operator(expm(-1j * H_SI * time))
    qc = QuantumCircuit(N_sites, 1)
    half_N_sites = N_sites // 2  # Calculate the size of the first half of the chain
    qc.x(range(half_N_sites))  # Apply X gate to the  first half of the qubits i.e. first and second qubits to create |1100> state for 4 N_sites
    qc.unitary(U, range(N_sites), label="exp(-iHt)")
    qc.measure(0, 0)  # Measure all qubits
    return qc

# Generate and visualize the circuit for a specific time
qc = evolve_and_measure_circuit(τ , H_SI) # τ  used here to see the qc at each time step
qc.draw(output='mpl', style='clifford')

# Simulation and expected value calculation
sigma_z_values = []
survival_probabilities = []

for t in times:
    qc = evolve_and_measure_circuit(t, H_SI)
    result = execute(qc, backend, shots=shots).result()
    counts = result.get_counts(qc)

    # Calculate expectation value of sigma_z for the first qubit
    sigma_z = 0
    for outcome, count in counts.items():
        if outcome[0] == '0':
            sigma_z += count / shots
        else:
            sigma_z -= count / shots
    
    # Calculate survival probability
    survival_probability = (1 - sigma_z) / 2
    
    # Append values to the respective lists
    sigma_z_values.append(sigma_z)
    survival_probabilities.append(survival_probability)


def find_first_local_minima_index(arr):
    n = len(arr)
    for i in range(1, n - 1):
        if arr[i] < arr[i - 1] and arr[i] < arr[i + 1]:
            return i
    return -1

# Index of the first minimum of the survival probabilities
i_first_local_min = find_first_local_minima_index(survival_probabilities)

# Writing if_else statement to communicate if local minima (not) found
if i_first_local_min != -1:
    print(f"Index of the first local minimum: {i_first_local_min}")
else:
    print("No local minimum found in the array.")

# Time at which the first minimum survival probability is reached
t_min = τ * i_first_local_min - τ
print(f"Corresponding time of first minimum index= {t_min}")

# Calculate Rogerro's fit for the first minimum of the survival probability
a_t = 0
b_t = 2.105
c_t = 0

t_p_Rog = a_t * np.log(N_sites) + b_t * np.sqrt(N_sites) + c_t
print("t_p_Rog= ", t_p_Rog)

# Assert condition
# assert abs(t_min - t_p_Rog) < τ + tolerance, "The time of the first minimum survival probability is not within the expected range."

# Plot the survival probability at site 1 against time
plt.plot(times, survival_probabilities, label='Survival Probability')
plt.xlabel('Time')
plt.ylabel('Survival Probability at site 1')
plt.title('Survival Probability at site 1 vs Time (4 Qubits, All-to-All Interaction)')

# Add scatter point for Rogerro's fit
plt.scatter([t_p_Rog], [survival_probabilities[np.abs(times - t_p_Rog).argmin()]], color='red', label="t_p_Rog", zorder=5)

# Add scatter point for the first local minimum
if i_first_local_min != -1:
    plt.scatter([t_min], [survival_probabilities[np.abs(times - t_min).argmin()]], color='blue', label="First Local Min", zorder=5)

plt.legend(loc="upper right")
plt.grid(True)
plt.show()